In [ ]:
# Modules to import

!pip install --user nipy
!pip install --user nilearn
!pip install --user seaborn
!pip install --user keras

In [1]:
# Main Imports

import nilearn

from nilearn.image import resample_to_img

import pylab as plt

import numpy as np
import nibabel as nb
import os
import glob
import random
import pandas as pd
import re

import seaborn as sns #added
sns.set(style="darkgrid") #added

from nilearn.image import mean_img #added
from nilearn.plotting import plot_anat #added

# Principal code parameters

In [2]:
# Local directory where the Data are mounted
rootDirectory = "/data/RMN/LUCA_PASQUINI"

# Local subdirectory where dataset is mounted
dataDir = "DATI_SEGMENTATI_SCALATI_media"

# Dataset dir
datasetDir = f"{rootDirectory}/{dataDir}"

#CSV fileroot 
fileName = f"{rootDirectory}/{dataDir}/Array_Labels_Def.csv"

SUBJECT_NAME_REPEATED=["BIANCHI","BOVE","PROIETTI"]

#for i in range(len(SUBJECT_NAME_REPEATED)):
#    f"{SUBJECT_NAME_REPEATED[i]}"
#    print(f"{SUBJECT_NAME_REPEATED[i]}")

PATIENT_REMOVED=['Pascal','Mitchell','Rufini','Farella','Array']

SEQUENCE_1= "ADC_registered"
SEQUENCE_2= "ADC"
#T1_registered , T1
#T2_registered , T2
#FLAIR_registered , FLAIR
#ADC_registered , ADC
#rCBV_registered , rCBV
#MPRAGEMDC
MaskPath = "SOLID"
#SOLID
#NECROSI
#T1ROI
#T2ROI


#reference dimensions
dim1=192
dim2=256
dim3=144


'''Model parameters'''

#percentage of training test
p=0.8

# Specify shape of convolution kernel
kernel_size = (3, 3)

# Specify number of output categories
n_classes = 2

# Specify number of filters per layer
filters = 16

#added block
nEpochs = 100  # Increase this value for better results (i.e., more training)

batch_size = 16   # Increasing this value might speed up fitting

activation='relu'

activation_Dense='softmax'

# optimizer
learning_rate = 1e-5
#adam = Adam(lr=learning_rate)
#sgd = SGD(lr=learning_rate)

loss='categorical_crossentropy'

metrics=['accuracy']


# Survival Labels dataframe

In [ ]:
#CSV fileroot 
#fileName = "/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/Array_Labels_Def.csv"

#csv file
df = pd.read_csv(fileName, sep=";", header=None)

#Dataframe columns title
df.columns = ["Subject", "Survival"]

#Subject column as index
df = df.set_index('Subject')

#in order to check the entire dataframe of labels
pd.set_option('display.max_rows', len(df))
print(df)

# Functions

In [3]:
#NORMALIZE_NAME: the function takes the subject path basename and splits it according to the underscore
#in order to take the first string and use it as index in dataframe.
#If a string is repeated, the function adds the first letter of the second string after the underscore

def normalize_name(subject_path, add_name):
    #Components= []
    subject = os.path.basename(subject_path)
    subject = re.sub(r'^(DE|D|DI|LO|DEL)_', '', subject)
    components =  subject.split("_")
    if add_name[0]==components[0] or add_name[1]==components[0] or add_name[2]==components[0] :
            return components[0].title() + components[1][0].capitalize()
    else:
            return components[0].title()
        
    
#GET_SUBJECT_METADATA:the function recives, as input, the subject path and the list of name repeated already known
#the output are the path base name (SURNAME_NAME) and the normalized name
#add_name needs to contain the list of name repeated


def get_subject_metadata(subject_path, subjects_with_name=[]):
    dirname = os.path.basename(subject_path)
    return (dirname, normalize_name(subject_path, add_name=[name for name in subjects_with_name]))

In [4]:
'''' Get index positions of value in dataframe '''
def getIndexes(dfObj, value):
    listOfPos = list()
    result = dfObj.isin([value]) # Get bool dataframe with True at positions where the given value exists
    seriesObj = result.any() # Get list of columns that contains the value
    columnNames = list(seriesObj[seriesObj == True].index)
    for col in columnNames: # Iterate over list of columns and fetch the rows indexes where value exists
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
# Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

# Dataframe folders construction

In [ ]:
#Path and Subject columns
Df_Subjects_dirs = pd.DataFrame()
for subject_path in glob.glob(f"{datasetDir}/*"):
    subjects_dirs=[get_subject_metadata(subject_path , subjects_with_name=[SUBJECT_NAME for SUBJECT_NAME in SUBJECT_NAME_REPEATED])]
    print(subjects_dirs)
    df_subject_dirs = pd.DataFrame(subjects_dirs,columns=["Path", "Subject"])
    Df_Subjects_dirs=pd.concat([Df_Subjects_dirs,df_subject_dirs],ignore_index=True)


#in order to check the correct constucrion of 
pd.set_option('display.max_rows', len(Df_Subjects_dirs))
print(Df_Subjects_dirs)

#Subject as index
Df_Subjects_dirs = Df_Subjects_dirs.set_index('Subject')
pd.set_option('display.max_rows', len(Df_Subjects_dirs))

#in order to check the correct constucrion of Df_Subjects_dirs
print(Df_Subjects_dirs)

#Unnecessary strings removal with "drop" function
x=Df_Subjects_dirs
x.drop(index='Array', columns='Path')
# Delete rows with index label a & b    
modX= x.drop([PATIENT for PATIENT in PATIENT_REMOVED])
#modX["Path"]
modX = pd.DataFrame(modX)

#in order to check the correct construction of the dataframe
pd.set_option('display.max_rows', len(modX))
print(modX)

# Join function between Path and Labels

In [ ]:
result = modX.join(df, on='Subject')

#in order to check if all the dataframe is construced in the right way
pd.set_option('display.max_rows', len(result))
result   

# NAN removal

In [ ]:
#it takes the indexes to which the NaN corresponds
NAN_index=result['Survival'].index[result['Survival'].apply(np.isnan)]

#it takes the number of the corresponding row as int
df_index=result.index.values.tolist()
int_index=[df_index.index(i) for i in NAN_index]
#int_index

#than it is possible to obtain the corresponding survival label of the original array, df in this case
label=[df.iloc[x]['Survival'] for x in int_index]
label

#iteration to replace the NaNs
for l in label:
    result['Survival'].fillna(l,inplace=True)
    
#result is the final Dataframe with "Subject" as index and the columns "Path" and "Survival"
pd.set_option('display.max_rows', len(result))
result

In [5]:
import pickle
result = pickle.load( open( "DataFrame.pickle", "rb" ) )

# Matrix construction

In [6]:
#Data loading
listOfElems=[]  #array that need to obtain the effective subjects with the sequence that we want to analyse 
Dim = []  #array that need to contain the dimension of each image in order to check the most frequent or the minimum one
Data = [] #array that contains information of the image after the extraction of data from the nibabel format
IMG=[] #array that contains the "file.nii.gz" information thanks to the nibabel module 
for Path in result["Path"]: #here starts the iteration on all the paths written in the column "Path" in result dataframe
    #print(Path)
    if os.path.isfile(f"{datasetDir}/{Path}/{SEQUENCE_1}.nii"): #I need just the Paths that contain a certain sequence

        IMG_reg = nb.load(f"{datasetDir}/{Path}/{SEQUENCE_1}.nii") #nobabel module that allow the loading of nifty file
        DATA= np.asarray(IMG_reg.dataobj) #get_data takes the information about scale of gray
        a = [DATA.shape]
        Dim.append(a) #allows the array construction
        Data.append(DATA)  #allows the array construction
        IMG.append(IMG_reg)  #allows the array construction
        

        IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/{MaskPath}.nii") #that's the file of the mask that limits information only on a certain region of the tumor
        ROI_DATA=np.asarray(IMG_roi.dataobj)
        b=[ROI_DATA.shape]
        Dim.append(b)
        Data.append(ROI_DATA)
        IMG.append(IMG_roi)
        
        path=[f"{Path}"]
        listOfElems.append(path) #allows the array construction
        
    else: #anyway there are some patient with the sequence with different name but with the same information, so I can include them with the "else"
        if os.path.isfile(f"{datasetDir}/{Path}/{SEQUENCE_2}.nii"):
            IMG_reg = nb.load(f"{datasetDir}/{Path}/{SEQUENCE_2}.nii")
            DATA= np.asarray(IMG_reg.dataobj)
            a = [DATA.shape]
            Dim.append(a)
            Data.append(DATA)
            IMG.append(IMG_reg)
            
            IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/{MaskPath}.nii")
            ROI_DATA=np.asarray(IMG_roi.dataobj)
            b=[ROI_DATA.shape]
            Dim.append(b)
            Data.append(ROI_DATA)
            IMG.append(IMG_roi)
            
            path=[f"{Path}"]
            listOfElems.append(path)

In [ ]:
IMG_reg = nb.load(f"{datasetDir}/BIANCHI_ORAZIO/{SEQUENCE_1}.nii") #nibabel module that allow the loading of nifty file
DATA= np.asarray(IMG_reg.dataobj)






In [7]:
len(listOfElems)

114

In [ ]:
Data[20][39][100][30]

In [26]:
'''Labels Array Construction'''

#the following iteration allows the extraction of indexes corresponding to the paths selected by the preavious iterations
T1_Subject=[]
for i in range(len(listOfElems)):
    Pos=getIndexes(result, listOfElems[i][0])
    print(Pos[0][0])
    T1_Subject.append(Pos[0][0])  #the T1_Subject array contains these information

#the following iteration takes the corresponding Survival information on the result Dataframe built before    
T1_Subject_array=np.asarray(T1_Subject)

T1_Labels=[]
for i in range(len(T1_Subject_array)):
    lab = result.loc[T1_Subject_array[i],"Survival"]
    T1_Labels.append(int(lab)) #creates the list that contains these information

T1_Labels=np.asarray(T1_Labels) #that's the relative array
T1_Labels.shape

#the following iteration allows the construction of the final labels array with doubled information of each element of T1_Labels array
Label_Def=[]
for x in range(len(T1_Labels)):
    label_Def=[[T1_Labels[x]]*2]
    Label_Def.append(label_Def)
    
Label_Def=np.asarray(Label_Def)
Label_Def=np.ravel(Label_Def)
Label_Def #that's the final label array that can be used for the training

Alessandrini
Angeloni
Assanto
Bagnoli
Barontini
Battista
Bergnach
Bertuzzi
Bevilacqua
BianchiG
Boezi
Cacace
Caldaroni
Camacci
Camplese
Capezzone
Carzedda
Colafrancesco
Colamartini
Colazzo
Coletta
Cosimi
Crescenzi
Darida
BoveP
Santis
Carlatonio
Lorenzo
Maso
Massa
Dobrisan
Droghei
Angeli
Angelo
Fabiani
Federico
Ferrazza
Filipponi
Florio
Fratini
Gattamorta
Geggi
Gennari
Gianfelici
Gioia
Giordano
Inciti
Ionta
Isoni
Labella
Landone
Lioce
Longo
Bello
Lupi
Lupo
Maiolini
Maragno
Marcolini
Mariani
Marocchi
Martella
Martinez
Masci
Medici
Micheli
Monacelli
Moscardini
Musat
Nerone
Pagannone
Pagliaroli
Pagnotta
Palma
Palmieri
Panetti
Passari
Pieri
Pierini
Pineda
Pistoia
Podagrosi
Podda
Principi
ProiettiG
ProiettiM
Quattrociocchi
Re
Ricci
Romito
Rosari
Ruscito
Rusnac
Russo
Sala
Saltarelli
Santini
Soloviy
Stan
Stefanini
Sterpa
Sykula
Tavolucci
Tempestini
Temporin
Teofili
Testa
Tiberi
Tomao
Tomeo
Veronesi
Vitulano
Zangari
Zeppa


array([0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0])

In [9]:
'''Working on dimensions'''

#in order to work on dimensions it's necessary the array construction (as predicted in previous blocks)
Dim=np.asarray(Dim)   
Data=np.asarray(Data)
IMG=np.asarray(IMG)

#in order to know the minimum dimensions
Min_value=np.amin(Dim, axis=1)
Min_value
Min=np.amin(Min_value, axis=0)
Min_value.shape[0]

not_in_index = [k for k in range(Min_value.shape[0]) if not np.all(Min_value[k] == (dim1, dim2, dim3))] #that's the
#construction of an array that contains the position not corresponding to the dimension researched [(192,256,144)
#in this case]

pos_1=np.where(Min_value[:,0]==dim1) #position with first dimension equal to 192
pos_2=np.where(Min_value[:,1]==dim2) #position with first dimension equal to 256
pos_3=np.where(Min_value[:,2]==dim3) #position with first dimension equal to 144
eq=np.intersect1d(pos_1,pos_2)  #that command in order to find the intersection between the pos_1 and pos_2
index_IMG=np.intersect1d(eq,pos_3) #intersection that gives the complememntary information of not_in_index

#Here is given a random position that corresponds to the dimension request
def_index=random.choice(index_IMG)
print(def_index)


not_in_index

149


[0,
 1,
 2,
 3,
 8,
 9,
 10,
 11,
 12,
 13,
 16,
 17,
 18,
 19,
 20,
 21,
 24,
 25,
 26,
 27,
 32,
 33,
 34,
 35,
 36,
 37,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 68,
 69,
 74,
 75,
 84,
 85,
 88,
 89,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 102,
 103,
 104,
 105,
 106,
 107,
 110,
 111,
 114,
 115,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 134,
 135,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 150,
 151,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227]

In [10]:
'''RESAMPLE BLOCK'''

#The resample function is executed only on images without the dimension request, respect to a random image with dimension (192,256,144)
for i in not_in_index:
    Res=nilearn.image.resample_to_img(IMG[i], IMG[def_index],interpolation='nearest')
    IMG[i]=Res
    Data[i]=np.asarray(IMG[i].dataobj)

In [11]:
import pickle
pickle.dump( Data[:], open( "ADC_Solid_Dataset.pickle", "wb" ) )

In [ ]:
#np.save('rCBV_Solid', Data)

#Data=np.load('rCBV_Solid.npy',allow_pickle=True )

In [12]:
Input_matrix=np.array(Data[:])

In [13]:
Input_matrix.shape

(228,)

In [14]:
'''Reshaping of Input Matrix'''

Input_matrix=np.empty((len(Data),dim1,dim2,dim3)) #in order to generate an empty array with a fixed shape

for i in not_in_index:

    Input_matrix[i,:,:,:]=np.array(Data[i])




for i in index_IMG:

    Input_matrix[i,:,:,:]=np.array(Data[i])
    
    
#in order to check the correct construction    
Input_matrix.shape

#import pickle
#pickle.dump( Input_matrix, open( "Input_matrix.pickle", "wb" ) )

(228, 192, 256, 144)

In [22]:


data = Input_matrix[...,None]
data.shape



(228, 192, 256, 144, 1)

# TRAINING

In [15]:
'''Modules needed'''

import tensorflow as tf
from keras.models import Sequential

from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import regularizers

from keras.optimizers import Adam, SGD

from keras import backend as K

Using TensorFlow backend.


In [16]:
labels=Label_Def
from keras.utils import to_categorical
labels = to_categorical(labels)

labels

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [ ]:
'''## Create list of indices and shuffle them
N = Input_matrix.shape[0]
indices = np.arange(N)
np.random.shuffle(indices)

#  Cut the dataset at 80% to create the training and test set
N_80p = int(p * N)
indices_train = indices[:N_80p]
indices_test = indices[N_80p:]

# Split the data into training and test sets
X_train = Input_matrix[indices_train, ...]
X_test = Input_matrix[indices_test, ...]

print(X_train.shape, X_test.shape)

labels=Label_Def

#Outcome variable block added
y_train = labels[indices_train] == 0
y_test = labels[indices_test] == 1

from keras.utils import to_categorical #Convert a class vector (integer) into binary class matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train'''

In [17]:
'''Parameters setting'''

# Get shape of input data
data_shape = Data[0].shape

data_shape

(192, 256, 144)

# Model

In [18]:
#model block added
K.clear_session()
model = Sequential()

model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=data_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 2, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 4, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

# optimizer
learning_rate = 1e-5
adam = Adam(lr=learning_rate)
sgd = SGD(lr=learning_rate)

model.compile(loss='binary_crossentropy',
              optimizer=adam, # swap out for sgd 
              metrics=['accuracy','binary_crossentropy'])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 190, 254, 16)      20752     
_________________________________________________________________
batch_normalization_1 (Batch (None, 190, 254, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 95, 127, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 93, 125, 32)       4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 93, 125, 32)       128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 46, 62, 32)        0         
______________________________________

In [19]:
#model block added
K.clear_session()
model = Sequential()

model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=data_shape,kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 2, kernel_size, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 4, kernel_size, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.001)))

# optimizer
learning_rate = 1e-5
adam = Adam(lr=learning_rate)
sgd = SGD(lr=learning_rate)

model.compile(loss='binary_crossentropy',
              optimizer=adam, # swap out for sgd 
              metrics=['accuracy','binary_crossentropy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 190, 254, 16)      20752     
_________________________________________________________________
batch_normalization_1 (Batch (None, 190, 254, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 95, 127, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 93, 125, 32)       4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 93, 125, 32)       128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 46, 62, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 44, 60, 64)       

In [ ]:
import tensorflow as tf

base_model = tf.keras.applications.VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)
base_model.trainable = False
model = base_model.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dense(units=512, activation='relu')(model)
model = tf.keras.layers.Dropout(0.7)(model)
predictions = tf.keras.layers.Dense(units=2, activation='softmax')(model)
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
'''Model fitting'''
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
# patient early stopping
#es = EarlyStopping(monitor='accuracy', mode='max', patience=100)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)



%time fit = model.fit(Input_matrix, labels, epochs=500,callbacks=[callback],validation_split=0.2, shuffle=True,batch_size=batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 182 samples, validate on 46 samples
Epoch 1/500
182/182 [==============================] - 26s 142ms/step - loss: 1.9880 - accuracy: 0.5220 - binary_crossentropy: 1.0387 - val_loss: 1.7530 - val_accuracy: 0.5217 - val_binary_crossentropy: 0.8038
Epoch 2/500
182/182 [==============================] - 19s 102ms/step - loss: 1.9917 - accuracy: 0.5220 - binary_crossentropy: 1.0425 - val_loss: 1.6942 - val_accuracy: 0.5435 - val_binary_crossentropy: 0.7451
Epoch 3/500
182/182 [==============================] - 19s 103ms/step - loss: 1.9408 - accuracy: 0.5275 - binary_crossentropy: 0.9916 - val_loss: 1.6713 - val_accuracy: 0.5435 - val_binary_crossentropy: 0.7223
Epoch 4/500
182/182 [==============================] - 18s 100ms/step - loss: 1.9497 - accuracy: 0.5220 - binary_crossentropy: 1.0006 - val_loss: 1.6891 - val_accuracy: 0.5435 - val_binary_crossentropy: 0.7401
Epoch 5/500
182/182 

182/182 [==============================] - 17s 95ms/step - loss: 1.3368 - accuracy: 0.7363 - binary_crossentropy: 0.3942 - val_loss: 1.7541 - val_accuracy: 0.5652 - val_binary_crossentropy: 0.8115
Epoch 77/500
182/182 [==============================] - 17s 93ms/step - loss: 1.3344 - accuracy: 0.6868 - binary_crossentropy: 0.3919 - val_loss: 1.7556 - val_accuracy: 0.5652 - val_binary_crossentropy: 0.8131
Epoch 78/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3525 - accuracy: 0.7363 - binary_crossentropy: 0.4101 - val_loss: 1.7392 - val_accuracy: 0.5870 - val_binary_crossentropy: 0.7969
Epoch 79/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3538 - accuracy: 0.7582 - binary_crossentropy: 0.4115 - val_loss: 1.7873 - val_accuracy: 0.5870 - val_binary_crossentropy: 0.8451
Epoch 80/500
182/182 [==============================] - 17s 93ms/step - loss: 1.3543 - accuracy: 0.7253 - binary_crossentropy: 0.4122 - val_loss: 1.7662 - val_accuracy: 0.54

182/182 [==============================] - 17s 95ms/step - loss: 1.3278 - accuracy: 0.7088 - binary_crossentropy: 0.3902 - val_loss: 1.6978 - val_accuracy: 0.5870 - val_binary_crossentropy: 0.7603
Epoch 116/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3331 - accuracy: 0.7143 - binary_crossentropy: 0.3956 - val_loss: 1.7006 - val_accuracy: 0.5435 - val_binary_crossentropy: 0.7632
Epoch 117/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3262 - accuracy: 0.6813 - binary_crossentropy: 0.3889 - val_loss: 1.7063 - val_accuracy: 0.5435 - val_binary_crossentropy: 0.7691
Epoch 118/500
182/182 [==============================] - 17s 95ms/step - loss: 1.3471 - accuracy: 0.7308 - binary_crossentropy: 0.4099 - val_loss: 1.7106 - val_accuracy: 0.5435 - val_binary_crossentropy: 0.7735
Epoch 119/500
182/182 [==============================] - 17s 96ms/step - loss: 1.3197 - accuracy: 0.7363 - binary_crossentropy: 0.3826 - val_loss: 1.7387 - val_accuracy: 

Epoch 154/500
182/182 [==============================] - 17s 93ms/step - loss: 1.3292 - accuracy: 0.7033 - binary_crossentropy: 0.3976 - val_loss: 1.8158 - val_accuracy: 0.5217 - val_binary_crossentropy: 0.8843
Epoch 155/500
182/182 [==============================] - 17s 93ms/step - loss: 1.3366 - accuracy: 0.7582 - binary_crossentropy: 0.4052 - val_loss: 1.8649 - val_accuracy: 0.5217 - val_binary_crossentropy: 0.9335
Epoch 156/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3417 - accuracy: 0.7418 - binary_crossentropy: 0.4104 - val_loss: 1.9095 - val_accuracy: 0.4783 - val_binary_crossentropy: 0.9783
Epoch 157/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3423 - accuracy: 0.7802 - binary_crossentropy: 0.4112 - val_loss: 1.9049 - val_accuracy: 0.5000 - val_binary_crossentropy: 0.9739
Epoch 158/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3198 - accuracy: 0.7198 - binary_crossentropy: 0.3888 - val_loss: 1.8095 - 

Epoch 193/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3103 - accuracy: 0.7198 - binary_crossentropy: 0.3854 - val_loss: 1.8077 - val_accuracy: 0.5435 - val_binary_crossentropy: 0.8829
Epoch 194/500
182/182 [==============================] - 17s 94ms/step - loss: 1.2965 - accuracy: 0.7802 - binary_crossentropy: 0.3717 - val_loss: 1.8179 - val_accuracy: 0.5870 - val_binary_crossentropy: 0.8932
Epoch 195/500
182/182 [==============================] - 17s 95ms/step - loss: 1.2845 - accuracy: 0.7527 - binary_crossentropy: 0.3599 - val_loss: 1.8249 - val_accuracy: 0.5000 - val_binary_crossentropy: 0.9004
Epoch 196/500
182/182 [==============================] - 17s 93ms/step - loss: 1.3085 - accuracy: 0.7308 - binary_crossentropy: 0.3841 - val_loss: 1.8300 - val_accuracy: 0.5000 - val_binary_crossentropy: 0.9057
Epoch 197/500
182/182 [==============================] - 17s 94ms/step - loss: 1.3022 - accuracy: 0.7582 - binary_crossentropy: 0.3780 - val_loss: 1.8508 - 

Epoch 232/500
182/182 [==============================] - 17s 95ms/step - loss: 1.2940 - accuracy: 0.7418 - binary_crossentropy: 0.3764 - val_loss: 1.9137 - val_accuracy: 0.5000 - val_binary_crossentropy: 0.9962
Epoch 233/500
182/182 [==============================] - 17s 95ms/step - loss: 1.2988 - accuracy: 0.7418 - binary_crossentropy: 0.3814 - val_loss: 1.8929 - val_accuracy: 0.5000 - val_binary_crossentropy: 0.9756
Epoch 234/500
182/182 [==============================] - 17s 93ms/step - loss: 1.2940 - accuracy: 0.7198 - binary_crossentropy: 0.3768 - val_loss: 1.8735 - val_accuracy: 0.5217 - val_binary_crossentropy: 0.9564
Epoch 235/500
182/182 [==============================] - 17s 95ms/step - loss: 1.2899 - accuracy: 0.7418 - binary_crossentropy: 0.3729 - val_loss: 1.8680 - val_accuracy: 0.5217 - val_binary_crossentropy: 0.9512
Epoch 236/500
182/182 [==============================] - 17s 94ms/step - loss: 1.2979 - accuracy: 0.7527 - binary_crossentropy: 0.3811 - val_loss: 1.8991 - 

Epoch 271/500
182/182 [==============================] - 17s 96ms/step - loss: 1.2851 - accuracy: 0.7253 - binary_crossentropy: 0.3759 - val_loss: 1.9216 - val_accuracy: 0.4783 - val_binary_crossentropy: 1.0125
Epoch 272/500
182/182 [==============================] - 17s 93ms/step - loss: 1.2826 - accuracy: 0.7692 - binary_crossentropy: 0.3735 - val_loss: 1.9389 - val_accuracy: 0.4783 - val_binary_crossentropy: 1.0299
Epoch 273/500
182/182 [==============================] - 17s 93ms/step - loss: 1.2992 - accuracy: 0.7473 - binary_crossentropy: 0.3904 - val_loss: 1.8782 - val_accuracy: 0.5217 - val_binary_crossentropy: 0.9695
Epoch 274/500
182/182 [==============================] - 17s 94ms/step - loss: 1.2778 - accuracy: 0.7308 - binary_crossentropy: 0.3692 - val_loss: 1.8586 - val_accuracy: 0.5000 - val_binary_crossentropy: 0.9502
Epoch 275/500
182/182 [==============================] - 17s 93ms/step - loss: 1.2880 - accuracy: 0.7143 - binary_crossentropy: 0.3796 - val_loss: 1.8663 - 

In [ ]:
# evaluate the model
scores = model.evaluate(Input_matrix, labels, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[0]*100))


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_T1_Solid_Surv.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_T1_Solid_Surv.h5")
print("Saved model to disk")

# Model plot

In [ ]:
fig = plt.figure(figsize=(40, 25))
epoch = np.arange(202) + 1
fontsize = 30
plt.plot(epoch, fit.history['accuracy'], marker="o", linewidth=2,
         color="steelblue", label="train")
plt.plot(epoch, fit.history['val_accuracy'], marker="o", linewidth=2,
         color="red", label="test")
plt.plot(epoch, fit.history['loss'], marker="o", linewidth=2,
         color="orange", label="loss")
plt.xlabel('Epoch', fontsize=fontsize)
plt.ylabel('% Accuracy', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(frameon=False, fontsize=30);
